<a href="https://colab.research.google.com/github/perurianusha/git-repo/blob/master/SHL_Assessment_Recommendation_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://www.shl.com/solutions/products/product-catalog/"  # Placeholder

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Example: loop through assessment cards (change tags/classes as per actual HTML)
assessments = soup.find_all("div", class_="assessment-card")

for card in assessments:
    title = card.find("h3").text.strip()
    duration = card.find("span", class_="duration").text.strip()
    print(f"Assessment: {title}, Duration: {duration}")


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Set the URL
url = "https://www.shl.com/solutions/products/product-catalog/"

# Set a user-agent to mimic a browser
headers = {
    "User-Agent": "Mozilla/5.0"
}

# Send HTTP request
response = requests.get(url, headers=headers)

# Check for success
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all product containers (NOTE: class names may vary, inspect the site first)
    product_cards = soup.find_all("div", class_="product-item")

    data = []

    for card in product_cards:
        try:
            title = card.find("h3").text.strip()
        except:
            title = "N/A"

        try:
            description = card.find("div", class_="product-desc").text.strip()
        except:
            description = "N/A"

        try:
            link = card.find("a", href=True)["href"]
            if not link.startswith("http"):
                link = "https://www.shl.com" + link
        except:
            link = "N/A"

        # Save each product's info
        data.append({
            "Title": title,
            "Description": description,
            "Link": link
        })

    # Save to CSV
    df = pd.DataFrame(data)
    df.to_csv("shl_products.csv", index=False)
    print("✅ Data saved to 'shl_products.csv'")

else:
    print("❌ Failed to fetch page, status code:", response.status_code)


✅ Data saved to 'shl_products.csv'


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.shl.com/solutions/products/product-catalog/"
headers = {
    "User-Agent": "Mozilla/5.0"
}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    product_cards = soup.find_all("div", class_="product-item")

    data = []
    for card in product_cards:
        title = card.find("h3").text.strip() if card.find("h3") else "N/A"
        description = card.find("div", class_="product-desc").text.strip() if card.find("div", class_="product-desc") else "N/A"
        link_tag = card.find("a", href=True)
        link = "https://www.shl.com" + link_tag["href"] if link_tag else "N/A"

        data.append({
            "Title": title,
            "Description": description,
            "Link": link,
            "Job_Level": "Mid",         # Add dummy metadata for testing
            "Industry": "Tech",
            "Job_Title": "Software Engineer"
        })

    df = pd.DataFrame(data)
    df.to_csv("shl_products.csv", index=False)
    print("✅ Saved scraped data to shl_products.csv")
else:
    print("❌ Failed to fetch the SHL catalog.")


✅ Saved scraped data to shl_products.csv


In [8]:
with open("shl_products.csv", "r") as f:
    content = f.read()
    print(content if content else "❌ File is empty.")


In [9]:
import pandas as pd

data = [
    {
        "Title": "Cognitive Ability Test",
        "Description": "Tests general reasoning and logic skills",
        "Link": "https://shl.com/cognitive",
        "Job_Level": "Mid",
        "Industry": "Tech",
        "Job_Title": "Software Engineer"
    },
    {
        "Title": "Leadership Assessment",
        "Description": "Assesses leadership and strategic skills",
        "Link": "https://shl.com/leadership-assessment",
        "Job_Level": "Senior",
        "Industry": "Any",
        "Job_Title": "Manager"
    },
    {
        "Title": "Customer Service Test",
        "Description": "Measures communication and service skills",
        "Link": "https://shl.com/customer-service",
        "Job_Level": "Entry",
        "Industry": "Retail",
        "Job_Title": "Sales Associate"
    }
]

df = pd.DataFrame(data)
df.to_csv("shl_products.csv", index=False)
print("✅ Manually saved mock product catalog to shl_products.csv")


✅ Manually saved mock product catalog to shl_products.csv


In [13]:
df = pd.read_csv("shl_products.csv")

def recommend_assessments(job_title, job_level, industry):
    filtered_df = df[
        (df["Job_Title"].str.lower().str.contains(job_title.lower())) &
        (df["Job_Level"].str.lower() == job_level.lower()) &
        (df["Industry"].str.lower() == industry.lower())
    ]
    if filtered_df.empty:
        filtered_df = df[
            (df["Job_Level"].str.lower() == job_level.lower()) |
            (df["Industry"].str.lower() == industry.lower())
        ]
    return filtered_df[["Title", "Description", "Link"]]

job_title = input("Enter job title: ")
job_level = input("Enter job level (Entry/Mid/Senior): ")
industry = input("Enter industry: ")

recommendations = recommend_assessments(job_title, job_level, industry)

if not recommendations.empty:
    print("\nRecommended SHL Assessments:")
    print(recommendations.to_string(index=False))
else:
    print("No relevant assessments found.")


Enter job title: Data enginer
Enter job level (Entry/Mid/Senior): mid
Enter industry: tech

Recommended SHL Assessments:
                 Title                              Description                      Link
Cognitive Ability Test Tests general reasoning and logic skills https://shl.com/cognitive
